In [ ]:
import numpy as np, torch, json, time, os, igl
from Ctubes.geometry_utils import regular_polygon
from Ctubes.plot_utils import plot_generatrix
from Ctubes.tubes import Directrix, Generatrix, CTube
from Ctubes.opt import CTubeOptimizationProblem
from Ctubes.misc_utils import get_pairings_exact, load_curve_from_obj
from Ctubes.path_utils import get_name, setup_paths

paths = setup_paths(get_name())

PI = np.pi
TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)
torch.set_printoptions(precision=4)

%load_ext autoreload
%autoreload 2

%matplotlib widget

# Figure 17: Tangency to Minimal Surface

## #1: Pinned Ridge (Reference)

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig17_min_surf_winfty")

In [ ]:
# Generatrix: regular N-gon
N = 3
tube_radius = 0.3
generatrix_2d = regular_polygon(N, tube_radius)

# Translate cross-section s.t. the first vertex is at the origin
generatrix_2d = generatrix_2d - generatrix_2d[0]

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# Directrix: read curve from file
cps_ref = load_curve_from_obj(os.path.join(paths["data"], "fig17_min_surf/directrix_Q_high_res.obj"))
K = cps_ref.shape[0]
M = K

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

tube = CTube(directrix, generatrix)

In [ ]:
# Load surface from obj file
obj_file = os.path.join(paths["data"], f"fig17_min_surf/min_surf.obj")
V_surf, F_surf = igl.read_triangle_mesh(obj_file)

V_surf = torch.tensor(V_surf, dtype=TORCH_DTYPE)
F_surf = torch.tensor(F_surf, dtype=torch.int64)

directrix_target_surf_boundary = tube.directrix.clone()

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
target_surface = {
    'vertices': V_surf,
    'faces': F_surf
}

In [ ]:
# Set up optimization problem

opt_weights = {
    'join_ends': 1e1 / generatrix.aabb_diagonal_length() ** 2,
    'constrain_tube_ridges_to_surface': 1e0 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'join_ends_pairings': get_pairings_exact(N),
    'target_surface': target_surface,
    'constrained_tube_ridges': [0, 1]
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(False)
opt_prob.activate_theta(True)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(False)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

### Generate target fitting data for rendering 

In [ ]:
# Compute the distance of the constrained ridges from the target surface
from Ctubes.geometry_utils import point_mesh_squared_distance
ctube_vertices = opt_prob.tube_network.tubes[0].ctube_vertices
V_surf = opt_prob.objective_args['target_surface']['vertices']
F_surf = opt_prob.objective_args['target_surface']['faces']

constr_ridge_indices = [0, 1]
n_constr_ridges = len(constr_ridge_indices)
constrained_vertices = ctube_vertices[:, constr_ridge_indices]
pts_surf_squared_dist = torch.zeros((M, n_constr_ridges))
for i in range(M):
    for j in range(n_constr_ridges):
        pts_surf_squared_dist[i, j] += point_mesh_squared_distance(constrained_vertices[i, j].reshape(-1, 3), V_surf, F_surf)

# Format data, save json
data = {}
for j, ridge_i in enumerate(constr_ridge_indices):
    data[f"points_ridge_{ridge_i}"] = constrained_vertices[:, j].detach().numpy().tolist()
    data[f"surface_deviation_ridge_{ridge_i}"] = pts_surf_squared_dist[:, j].detach().numpy().tolist()
json_data = {}
json_data['surface_deviation'] = data

In [ ]:
# Save the data as json
file_path = os.path.join(paths["output_data"], "{}_target_deviation.json".format(paths["name"]))
with open(file_path, 'w') as f:
    json.dump(json_data, f, indent=4)

## #2: C-Tube Ridge Attraction to Surface Boundary

In [ ]:
# Generatrix: regular N-gon
N = 3
tube_radius = 0.3
generatrix_2d = regular_polygon(N, tube_radius)

# Translate cross-section s.t. the first vertex is at the origin
generatrix_2d = generatrix_2d - generatrix_2d[0]

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# Directrix: read curve from file
cps_ref = load_curve_from_obj(os.path.join(paths["data"], "fig17_min_surf/directrix_Q_low_res.obj"))
K = cps_ref.shape[0]
M = (K - 1) * 4 + 1

In [ ]:
# Load surface from obj file
obj_file = os.path.join(paths["data"], f"fig17_min_surf/min_surf.obj")
V_surf, F_surf = igl.read_triangle_mesh(obj_file)

V_surf = torch.tensor(V_surf, dtype=TORCH_DTYPE)
F_surf = torch.tensor(F_surf, dtype=torch.int64)

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

tube = CTube(directrix, generatrix)

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
target_surface = {
    'vertices': V_surf,
    'faces': F_surf
}

In [ ]:
w_preserve_curve = 1e3

In [ ]:
# Set up optimization problem

opt_weights = {
    'preserve_curve': w_preserve_curve / directrix.aabb_diagonal_length() ** 2,
    'preserve_tube_ridge_edge_directions': 1e4 / directrix.aabb_diagonal_length() ** 2,
    'smooth_plane_normal_diffs': 1e2,
    'join_ends': 1e3 / generatrix.aabb_diagonal_length() ** 2,
    'constrain_tube_ridges_to_surface': 1e0 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'join_ends_pairings': get_pairings_exact(N),
    'target_surface': target_surface,
    'constrained_tube_ridges': [0, 1],
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(True)
opt_prob.activate_theta(True)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(False)

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig17_min_surf_w{}".format(w_preserve_curve))

In [ ]:
# Set the target directrix equal to the boundary of the target surface
opt_prob.tube_network.tubes[0].directrix_ref = directrix_target_surf_boundary

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 500},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

### Generate target fitting data for rendering 

In [ ]:
# Compute the distance of the constrained ridges from the target surface
from Ctubes.geometry_utils import point_mesh_squared_distance
ctube_vertices = opt_prob.tube_network.tubes[0].ctube_vertices
V_surf = opt_prob.objective_args['target_surface']['vertices']
F_surf = opt_prob.objective_args['target_surface']['faces']

constr_ridge_indices = [0, 1]
n_constr_ridges = len(constr_ridge_indices)
constrained_vertices = ctube_vertices[:, constr_ridge_indices]
pts_surf_squared_dist = torch.zeros((M, n_constr_ridges))
for i in range(M):
    for j in range(n_constr_ridges):
        pts_surf_squared_dist[i, j] += point_mesh_squared_distance(constrained_vertices[i, j].reshape(-1, 3), V_surf, F_surf)

# Format data, save json
data = {}
for j, ridge_i in enumerate(constr_ridge_indices):
    data[f"points_ridge_{ridge_i}"] = constrained_vertices[:, j].detach().numpy().tolist()
    data[f"surface_deviation_ridge_{ridge_i}"] = pts_surf_squared_dist[:, j].detach().numpy().tolist()
json_data = {}
json_data['surface_deviation'] = data

In [ ]:
# Save the data as json
file_path = os.path.join(paths["output_data"], "{}_target_deviation.json".format(paths["name"]))
with open(file_path, 'w') as f:
    json.dump(json_data, f, indent=4)